In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
response = llm.invoke("Qual a previsão do tempo em Recife-RN?")
print(response.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a previsão do tempo. Recomendo que você consulte um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas sobre o tempo em Recife-RN.


In [6]:
a = 18273840567
b = 12830192835

response = llm.invoke("Quanto é 18273840567*12830192835? Responda apenas a o resultado.")
print(response.content)
print()
print(f'Resposta esperada:', a*b)

234250195202202999695.

Resposta esperada: 234456898310655737445


In [7]:
import random
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Soma dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a * b

@tool
def get_weather(city: str) -> str:
    """Retorna um clima aleatório para uma cidade.

    Args:
        city: Nome da cidade
    """
    weather = random.choice(["ensolarado", "nublado", "chuvoso", "tempestuoso", "neve"])
    return weather

tools = [add, multiply, get_weather]
llm_with_tools = llm.bind_tools(tools)

In [8]:
get_weather.invoke({"city": "Recife-RN"})

'neve'

In [9]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")]

ai_msg = llm_with_tools.invoke(messages)

print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'city': 'Recife-RN'}, 'id': 'call_LiuExQDva6hAj7NGkVUmsD5l', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 123, 'b': 321}, 'id': 'call_yLeMIhWXg0VWuCfYVH7GMGn4', 'type': 'tool_call'}]


In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [13]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")
    ]
})

print(response["messages"])

[HumanMessage(content='Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?', additional_kwargs={}, response_metadata={}, id='c15b98da-3054-45ba-8fa6-8cea28893125'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YgWKih1jJCTsti39t5m1XFg5', 'function': {'arguments': '{"city": "Recife-RN"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_XDnCSvvndvR4hpxjncuf6RSJ', 'function': {'arguments': '{"a": 123, "b": 321}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 145, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d5fc003d-dba4-4a8d-8225-ec577cd3

In [14]:
print(response["messages"][-1].content)

A previsão do tempo em Recife-RN é de céu nublado. 

Além disso, o resultado de 123 * 321 é 39.483.


## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [20]:
import os

@tool
def create_file(file_name: str, content: str) -> str:
    """Cria um arquivo de texto com o conteúdo

    Args:
        file_name: Nome do arquivo
        content: Conteúdo do arquivo
    """
    with open(file_name, 'w') as file:
        file.write(content)
    return f'Arquivo "{file_name}" criado com sucesso!'

tools.append(create_file)
agent_executor = create_react_agent(llm, tools)
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Crie um arquivo hello_world.txt contendo o texto olá mundo'")
    ]
})
print(response["messages"])
print(response["messages"][-1].content)

[HumanMessage(content="Crie um arquivo hello_world.txt contendo o texto olá mundo'", additional_kwargs={}, response_metadata={}, id='2a2ca00a-7c12-498f-885f-11b7d01f28dd'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VnEmFmAnhj7DdjeK9L40D2Ru', 'function': {'arguments': '{"file_name":"hello_world.txt","content":"olá mundo"}', 'name': 'create_file'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 187, 'total_tokens': 211, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e8d823b-5898-43a6-b92d-8da038555f1e-0', tool_calls=[{'name': 'create_file', 'args': {'file_name': 'hello_world.txt', 'content': 'olá mund

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [21]:
import os

@tool
def char_count(word: str, char: str) -> str:
    """Conta a quantidade de ocorrências de uma letra em uma palavra

    Args:
        word: palavra
    """
    return word.count(char)

tools.append(char_count)
agent_executor = create_react_agent(llm, tools)
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Quantas letras a existem na palavra avião?")
    ]
})
print(response["messages"])
print(response["messages"][-1].content)

[HumanMessage(content='Quantas letras a existem na palavra avião?', additional_kwargs={}, response_metadata={}, id='1329d166-a466-475f-8b3e-8fbfc38f312e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nc2ETiFMLIfeq6PU8Bvfeg99', 'function': {'arguments': '{"word":"avião","char":"a"}', 'name': 'char_count'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 220, 'total_tokens': 240, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-15ce7425-131b-4acd-906c-eeccc6871de3-0', tool_calls=[{'name': 'char_count', 'args': {'word': 'avião', 'char': 'a'}, 'id': 'call_nc2ETiFMLIfeq6PU8Bvfeg99', 'type': 'tool_call'}], usag